In [1]:
import requests
from bs4 import BeautifulSoup

### Web scraping the bigfoot website

In [2]:
URL = 'https://www.bfro.net/GDB/show_report.asp?id=1257'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')
features = [item.get_text() for item in soup.select("span.field")]
sighting_d = dict()
for item in features:
    if item == 'STATE:' or item == 'COUNTY:':
        sighting_d[item] = soup.find("span", text=item).next_element.next_element.next_element.next_element
    else:
        sighting_d[item] = soup.find("span", text=item).next_sibling

In [3]:
sighting_d

{'Submitted  by  witness   on Sunday,\xa0May\xa031,\xa01998.': '\n',
 'Family on their way home see a large, hairy creature cross the road': '\n',
 'YEAR:': ' 1998',
 'SEASON:': ' Spring',
 'MONTH:': ' May',
 'DATE:': ' 22',
 'STATE:': 'Alaska',
 'COUNTY:': 'Fairbanks County',
 'LOCATION DETAILS:': ' Fort Wainwright',
 'OBSERVED:': " I personally did not see it but a Noncommissioned officer i work with along with his wife, child and hunting buddy were on their way home when according to them a large, hairy about seven foot ape looking thing crossed the road in front of them. From what i could gather none of them are too familiar with bigfoot information. Any way they say it crossed the road which is about thirty-five feet in width in four to five steps it seemed and disappeared into the brush on the other side which leads to a river called the Chena. Both of the guys have been hunting since childhood and are sure they know a bear when they see one. The thing crossed the road on its hin